In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

In [ ]:
from pathlib import Path
import os
import pprint
import typing

import numpy as np
import torch
import librosa
from torch.nn import functional as F
import matplotlib.pyplot as plt
import IPython.display as ipd

from wavenet import model, train, sample, audio, datasets, utils, viz, debug

In [ ]:
pp = pprint.PrettyPrinter(indent=2)

# Train on Dataset in a folder

In [ ]:
# this cell contains papermill tagged parameters
# they can be overriden by the cli when training:  
# papermill in.ipynb out.ipynb -p batch_norm True

batch_norm = False
learning_rate = 0.0044
finder = False
batch_size = 12
max_epochs = 2
with_all_chans = None
root_dir: typing.Optional[Path] = None
cache_dir: typing.Optional[Path] = None

In [ ]:
if root_dir:
    root_dir = Path(root_dir)
else:
    raise Exception('provide a root_dir to read from.')
    
if cache_dir:
    cache_dir = Path(cache_dir)

In [ ]:
p = model.HParams(
    embed_inputs=True, 
    n_audio_chans=1, 
    squash_to_mono=True,
    batch_norm=batch_norm
)

if with_all_chans:
    p = p.with_all_chans(with_all_chans)
    
pp.pprint(dict(p))

In [ ]:
tp = train.HParams(
    max_epochs=max_epochs, 
    batch_size=batch_size, 
    num_workers=8, 
    finder=finder, 
    learning_rate=learning_rate
)

pp.pprint(dict(tp))

In [ ]:
utils.seed(p)
ds_train = datasets.Tracks.from_dir(p, root_dir, cache_dir)
ds_train

In [ ]:
n_examples = len(ds_train)
n_steps = tp.n_steps(n_examples)
n_examples, n_steps

In [ ]:
utils.seed(p)
m = model.Wavenet(p)
debug.summarize(m)

In [ ]:
t = train.Trainer(m, ds_train, None, tp, None)
t.metrics

In [ ]:
track_i = viz.plot_random_track(ds_train)
track, *_ = ds_train[track_i]
ipd.Audio(audio.mu_expand(track.squeeze().numpy(), p), rate=p.sampling_rate)

In [ ]:
%%capture
utils.seed(p)
t.train()

In [ ]:
utils.seed(p)
tracks, logits, g = sample.fast(m, ds_train.transforms, utils.decode_nucleus(), n_samples=32000, batch_size=10)

In [ ]:
for track in tracks:
    track = ds_train.transforms.normalise(track.numpy())
    track = audio.mu_expand(track, p)
    ipd.display(ipd.Audio(track, rate=p.sampling_rate))

In [ ]:
t.metrics.finish()